# 특징 기반 방법

아래의 코드로 그래프 기반 지도 머신 러닝을 해결하기 위한 매우 순진하지만 강력한 접근 방식을 탐구한다.
여기서는 사람이 만든 특징 추출의 고전적인 기계 학습 접근 방식에 의존한다.

1장에서 로컬 및 글로벌 그래프 속성을 그래프에서 추출하는 방법을 배웠다.
이러한 속성들은 그래프 분류에 유용할 수 있는 중요한 정보를 제공한다.

이번 데모에서는 StellarGraph에 내장된 PROTEINS 데이터 셋을 활용할 것이다.

In [7]:
from stellargraph import datasets
from IPython.display import display, HTML

dataset = datasets.PROTEINS()
display(HTML(dataset.description))
graphs, graph_labels = dataset.load()

각 그래프는 단백질을 나타내고 그래프 라벨은 효소인지 비효소인지를 나타낸다. 이 데이터셋에는 각 그래프에 대해 평균적으로 39개의 노드와 73개의 간선이 있는 1113개의 그래프가 있다. 그래프 노드에는 4개의 속성(라벨의 원-핫 인코딩 포함)이 있으며 각 그래프는 2개의 클래스 중 1개에 속하는 것으로 라벨이 지정된다.

그래프 측정 지표를 계산하기 위한 한 가지 방법은 각 그래프의 인접 행렬 표현을 찾는 것이다.

In [8]:
# StellarGraph 포맷을 numpy 인접 행렬 형태로 변환
adjs = [graph.to_adjacency_matrix().A for graph in graphs]
# 라벨을 Pandas.Series에서 Numpy 행렬로 변환
labels = graph_labels.to_numpy(dtype=int)

In [9]:
import numpy as np
import networkx as nx

metrics = []
for adj in adjs:
  G = nx.from_numpy_matrix(adj)
  # 기본 속성
  num_edges = G.number_of_edges()
  # 클러스터링 측정
  cc = nx.average_clustering(G)
  # 효율성 측정
  eff = nx.global_efficiency(G)

  metrics.append([num_edges, cc, eff])



이제 scikit-learn 유틸리티를 활용하여 훈련과 테스트셋을 생성할 수 있다. 실험에서는 데이터셋의 70%를 훈련 셋으로 사용하고 나머지를 테스트셋으로 사용한다.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(metrics, labels, test_size=0.3, random_state=42)

많은 기계 학습 워크플로에서 일반적으로 수행되는 것처럼 평균 및 단위 표준 편차가 0이 되도록 특징을 전처리한다.

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

이제 학습을 시작한다. 예제에는 서포트 벡터 머신(SVM)을 선택했다.

In [12]:
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

clf = svm.SVC()
clf.fit(X_train_scaled, y_train)

y_pred = clf.predict(X_test_scaled)

print('정확도', accuracy_score(y_test,y_pred))
print('정밀도', precision_score(y_test,y_pred))
print('재현율', recall_score(y_test,y_pred))
print('F1-score', f1_score(y_test,y_pred))

정확도 0.7455089820359282
정밀도 0.7709251101321586
재현율 0.8413461538461539
F1-score 0.8045977011494253
